In [5]:
import random
from collections import Counter
import itertools
import math

import networkx as nx
import numpy as np
import pandas as pd
from scipy.special import softmax
from tqdm import trange, tqdm

In [60]:
class Car:
    def __init__(self, source, target) -> None:
        self.source = source
        self.target = target

    def __repr__(self) -> str:
        return f'<Car {self.source} -> {self.target}>'

    def act(self, network):
        if self.source == self.target or not nx.has_path(network, self.source, self.target):
            return self.source
        else:
            choices = list(nx.all_shortest_paths(network, self.source, self.target, weight='anticipated_latency'))
            choice = random.choice(choices)
            print(f'Choose {choice} from {choices}.')
            return choice
            

class TrafficModel:
    def __init__(self, network, cars) -> None:
        self.network = network
        self.cars = cars

    def update_latencies(self):
        nx.set_edge_attributes(self.network, {(v, w): attr['latency_fn'](attr['flow']) for v, w, attr in self.network.edges(data=True)}, 'latency')
        nx.set_edge_attributes(self.network, {(v, w): attr['latency_fn'](attr['flow'] + 1) for v, w, attr in self.network.edges(data=True)}, 'anticipated_latency')

    def update_latency(self, edge):
        self.network.edges[edge]['latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['flow'])
        self.network.edges[edge]['anticipated_latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['flow'] + 1)

    @property
    def allowed_network(self):
        return nx.restricted_view(self.network, [], [(v, w) for v, w, allowed in self.network.edges(data='allowed') if not allowed])
    
    def decrease_flow(self, edge):
        self.set_flow(edge, self.network.edges[edge]['flow'] - 1)

    def increase_flow(self, edge):
        self.set_flow(edge, self.network.edges[edge]['flow'] + 1)

    def set_flow(self, edge, flow):
        self.network.edges[edge]['flow'] = flow
        self.network.edges[edge]['latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['flow'])
        self.network.edges[edge]['anticipated_latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['flow'] + 1)

In [73]:
def run(model, number_of_steps):
    routes = {id: [] for id in model.cars}

    nx.set_edge_attributes(model.network, 0, 'flow')
    model.update_latencies()

    stats = []
    for step in trange(number_of_steps):
        for id, car in np.random.permutation(list(model.cars.items())):
            for edge in zip(routes[id], routes[id][1:]):
                model.decrease_flow(edge)

            routes[id] = car.act(model.allowed_network)

            for edge in zip(routes[id], routes[id][1:]):
                model.increase_flow(edge)

        stats.append({
            **routes, 
            **nx.get_edge_attributes(model.network, 'latency'), 
            **{f'u({id})': nx.path_weight(model.network, route, 'latency') for id, route in routes.items()}})

    return pd.DataFrame(stats)

In [74]:
network = nx.DiGraph([(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)])

node_positions = {
    0: np.array([0, 0]),
    1: np.array([0.33, 0]),
    2: np.array([0.66, 0]),
    3: np.array([1, 0])
}

nx.set_node_attributes(network, node_positions, 'position')

# Latency is defined in terms of the load, i.e., the number of cars on the road
latency_fns = {
    (0, 0): lambda n: 1,
    (0, 1): lambda n: n,
    (0, 2): lambda n: 11,
    (1, 1): lambda n: 1,
    (1, 2): lambda n: 1,
    (1, 3): lambda n: 11,
    (2, 2): lambda n: 1,
    (2, 3): lambda n: n,
    (3, 3): lambda n: 1
}

nx.set_edge_attributes(network, latency_fns, 'latency_fn')

nx.set_edge_attributes(network, 0, 'flow')
nx.set_edge_attributes(network, 0, 'utilization')

nx.set_edge_attributes(network, True, 'allowed')

number_of_cars = 8
cars = {id: Car(0, 3) for id in range(number_of_cars)}

model = TrafficModel(network, cars)

In [75]:
run(model, 10)

100%|██████████| 10/10 [00:00<00:00, 2150.59it/s]

Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2, 3] from [[0, 1, 2, 3]].
Choose [0, 1, 2,

,0,1,2,3,4,5,6,7,"(0, 0)","(0, 1)",...,"(2, 3)","(3, 3)",u(0),u(1),u(2),u(3),u(4),u(5),u(6),u(7)
0,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
1,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
2,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
3,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
4,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
5,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
6,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
7,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
8,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
9,"[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]","[0, 1, 2, 3]",1,8,...,8,1,17,17,17,17,17,17,17,17
